In [1]:
from utils import Create_data, Create_text_embeddings, generate_patterns
import torch
import glob
import pandas as pd

c:\Users\ID\miniconda3\envs\pl\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
path_to_data = '../Datasets/Augmented_One_Particle_Dataset/Version_0/'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
df_labeled = pd.read_excel('../Datasets/dataset_labeled.xlsx')

In [3]:
pattern = None
path = path_to_data + 'Version_0'
with open('..\Datasets\procedure_patterns.txt') as f:
    lines = filter(None, (line.rstrip() for line in f))
    procedures = list(lines)
if pattern != None:
   procedures = [procedures[pattern]]
values = pd.read_excel('..\Datasets\{}'.format('synthesis_conditions_dataset.xlsx'))
df = pd.DataFrame(columns = ['image', 'text'])
df['image'] = pd.Series([file.replace(path_to_data + '/','') for file in glob.glob(path_to_data + '/*.jpg')])
labels = pd.DataFrame(columns=['Cube','Stick','Sphere','Flat','Amorphous'])
for index, row in enumerate(df.image):
    try:
      tmp = df_labeled[df_labeled.Image_id == int([x for x in row.split('_') if x.isdigit()][0])].reset_index().loc[0, 'Cube':'Amorphous']
      labels.loc[len(labels.index)] = tmp
      df.iloc[index, 1] = generate_patterns(procedures, values, int([x for x in row.split('_') if x.isdigit()][0]))[0]
    except:
       pass
df = df.dropna()

In [29]:
dataset_pattern_0 = pd.read_csv('text_dataset_pattern_0.csv')

In [32]:
dataset_pattern_0

,Flat,Amorphous,Sphere,Stick,Cube,text
0,0,0,0,1,1,All the materials were synthesized by the co-p...
1,0,0,0,1,0,All the materials were synthesized by the co-p...
2,0,0,1,0,0,All the materials were synthesized by the co-p...
3,0,0,1,0,1,All the materials were synthesized by the co-p...
4,0,0,0,1,1,All the materials were synthesized by the co-p...
...,...,...,...,...,...,...
204,0,0,0,0,1,All the materials were synthesized by the co-p...
205,0,1,0,0,1,All the materials were synthesized by the co-p...
206,0,0,0,1,1,All the materials were synthesized by the co-p...
207,0,0,0,1,1,All the materials were synthesized by the co-p...


In [33]:
Text_embeddings = Create_text_embeddings(dataset_pattern_0, device, batch_size=2000)

Batch 1 completed out of 1


In [6]:
Text_embeddings

tensor([[-1.0812e+00, -3.5154e-01, -3.3493e-04,  ..., -3.0808e-01,
          5.5263e-01,  5.9040e-01],
        [-1.0621e+00, -2.1866e-01, -1.0814e-02,  ..., -5.4173e-01,
          5.1261e-01,  6.6835e-01],
        [-8.7403e-01, -1.2175e-01,  8.8376e-02,  ..., -5.9882e-01,
          4.3596e-01,  5.9759e-01],
        ...,
        [-1.0363e+00, -1.6941e-01, -6.1533e-02,  ..., -3.6076e-01,
          4.2833e-01,  7.0775e-01],
        [-9.6085e-01, -2.2651e-01, -8.2822e-02,  ..., -4.3504e-01,
          4.1118e-01,  6.4225e-01],
        [-8.6137e-01, -2.7594e-02, -1.2861e-01,  ..., -6.4316e-01,
          4.4825e-01,  5.0638e-01]])

In [7]:
import pandas as pd
import numpy
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    precision_recall_curve,
)
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import warnings
from sklearn.model_selection import GridSearchCV
import pickle
import xgboost
warnings.filterwarnings("ignore")

In [20]:
def calculate_metrics(model, X_test, y_test):
    y_predict_proba = model.predict_proba(X_test)[:, 1]
    precision, recall, thresholds2 = precision_recall_curve(y_test, y_predict_proba)
    threshold = thresholds2[np.argmin(abs(precision - recall))]

    y_pred = (model.predict_proba(X_test)[:, 1] >= threshold).astype(int)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="macro")
    return acc, f1

In [25]:
X = Text_embeddings
y = labels['Flat']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, shuffle=True
)

rf_def = RandomForestClassifier(random_state=42)
rf_def.fit(X_train, y_train)
rf_def_acc, rf_def_f1 = calculate_metrics(rf_def, X_test, y_test)

In [27]:
rf_def_acc

0.905982905982906

In [28]:
rf_def_f1

0.6689814814814815